<a href="https://colab.research.google.com/github/dFarrulla/emd-desafio-junior-data-scientist/blob/main/analise_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import das Bibliotecas

In [ ]:
import requests
import json
from datetime import datetime as dt
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import streamlit as st



## Questões

#### 1. **Quantos feriados há no Brasil em todo o ano de 2024?**

In [ ]:
# Define a URL para a API de feriados públicos
url = 'https://date.nager.at/api/v3/publicholidays/2024/BR'

try:
    resposta = requests.get(url)
    resposta.raise_for_status()  # Levanta uma exceção para códigos de status HTTP ruins
    feriados = resposta.json()

    if isinstance(feriados, list):
        print(f"Temos {len(feriados)} feriados em 2024 no Brasil.")
    else:
        print("Formato inesperado de dados.")
except requests.exceptions.RequestException as e:
    print(f"Erro na requisição: {e}")

Temos 14 feriados em 2024 no Brasil.


R: Há 14 feriados durante o ano de 2024 no Brasil.

#### 2. **Qual mês de 2024 tem o maior número de feriados?**

In [ ]:
# Inicializa um contador para os meses
feriados_por_mes = Counter()

# Itera sobre os feriados, incrementando a contagem para o mês correspondente
for feriado in feriados:
    mes = feriado.get('date', '')[5:7]  # Extrai o mês da data
    if mes:
        feriados_por_mes[mes] += 1
    else:
        print(f"Aviso: Data inválida em '{feriado}'")

# Exibe o resultado
print(dict(feriados_por_mes))

{'01': 1, '02': 2, '03': 2, '04': 1, '05': 2, '07': 1, '09': 1, '10': 1, '11': 2, '12': 1}


R: Dois feriados em cada mês. Fevereiro, Março, Maio e Novembro.

#### 3. **Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?**

In [ ]:
feriados_em_dia_de_semana = sum(1 for feriado in feriados if verifica_dia_da_semana(feriado['date']))

print(f"Há {feriados_em_dia_de_semana} feriados em dias de semana.")

Há 9 feriados em dias de semana.


Há 9 feriados em dias de semana.

#### 4. **Qual foi a temperatura média em cada mês?**  
   Utilize a Open-Meteo Historical Weather API para obter as temperaturas médias diárias no Rio de Janeiro de 01/01/2024 a 01/08/2024.
   
   Verificando nas fontes, o Rio de Janeiro corresponde à:
   - Latitude: -22,9064; e
   - Longitude: -43,1822.

In [ ]:
# Coordenadas e URL da API
latitude, longitude = -22.9064, -43.1822
url = f"https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date=2024-01-01&end_date=2024-08-01&daily=temperature_2m_max&temperature_unit=celsius&timezone=America/Sao_Paulo"

# Faz a requisição GET à API
resposta = requests.get(url)

if resposta.ok:  # Verifica se a requisição foi bem-sucedida
    dados = resposta.json()

    # Dicionário para armazenar as temperaturas mensais
    temperaturas_mensais = defaultdict(list)

    # Itera sobre as datas e temperaturas, agrupando por mês
    for data, temp_max in zip(dados['daily']['time'], dados['daily']['temperature_2m_max']):
        mes = data[:7]  # Extrai o mês da data no formato "YYYY-MM"
        temperaturas_mensais[mes].append(temp_max)

    # Calcula e imprime a temperatura média de cada mês
    for mes, temps in temperaturas_mensais.items():
        print(f"Temperatura média em {mes}: {sum(temps) / len(temps):.2f}°C")
else:
    print(f"Erro ao acessar a API: {resposta.status_code}")

Temperatura média em 2024-01: 30.75°C
Temperatura média em 2024-02: 31.57°C
Temperatura média em 2024-03: 30.67°C
Temperatura média em 2024-04: 30.18°C
Temperatura média em 2024-05: 30.17°C
Temperatura média em 2024-06: 28.64°C
Temperatura média em 2024-07: 25.79°C
Temperatura média em 2024-08: 26.70°C


   
#### 5. **Qual foi o tempo predominante em cada mês nesse período?**  
   Utilize como referência para o código de tempo (_weather_code_) o seguinte link: [WMO Code](https://gist.github.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c).

In [ ]:
# Coordenadas do Rio de Janeiro
latitude = -22.9064
longitude = -43.1822

# URL da API Open-Meteo
url = f"https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date=2024-01-01&end_date=2024-08-01&daily=weathercode&timezone=America/Sao_Paulo"

# Faz a requisição GET à API
resposta = requests.get(url)

# Verifica se a requisição foi bem-sucedida
if resposta.status_code == 200:
    dados = resposta.json()

    # Dicionário para armazenar os códigos de tempo por mês
    weather_codes_mensais = defaultdict(list)

    # Itera sobre as datas e códigos de tempo diários
    for i in range(len(dados['daily']['time'])):
        data = dados['daily']['time'][i]
        weather_code = dados['daily']['weathercode'][i]

        # Extrai o mês da data
        mes = dt.strptime(data, "%Y-%m-%d").strftime("%Y-%m")

        # Adiciona o código de tempo à lista correspondente ao mês
        weather_codes_mensais[mes].append(weather_code)

    # Determina o tempo predominante em cada mês
    for mes, codes in weather_codes_mensais.items():
        # Conta a frequência de cada código de tempo no mês
        frequencia = Counter(codes)
        # Encontra o código de tempo mais comum
        tempo_predominante = frequencia.most_common(1)[0][0]
        print(f"Tempo predominante em {mes}: WMO Code {tempo_predominante}")
else:
    print(f"Erro ao acessar a API: {resposta.status_code}")

Tempo predominante em 2024-01: WMO Code 63
Tempo predominante em 2024-02: WMO Code 51
Tempo predominante em 2024-03: WMO Code 51
Tempo predominante em 2024-04: WMO Code 3
Tempo predominante em 2024-05: WMO Code 51
Tempo predominante em 2024-06: WMO Code 1
Tempo predominante em 2024-07: WMO Code 3
Tempo predominante em 2024-08: WMO Code 2


In [ ]:
# Coordenadas do Rio de Janeiro
latitude = -22.9064
longitude = -43.1822

# URL da API Open-Meteo
url = f"https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date=2024-01-01&end_date=2024-08-01&daily=weathercode&timezone=America/Sao_Paulo"

# Tabela de códigos WMO
wmo_codes = {
 0: "Céu limpo",
    1: "Céu limpo com poucas nuvens",
    2: "Parcialmente nublado",
    3: "Nublado",
    4: "Nublado com chuvas leves",
    5: "Chuva moderada",
    6: "Chuva forte",
    7: "Chuva intensa com trovoadas",
    8: "Neve leve",
    9: "Neve moderada",
    10: "Neve forte",
    11: "Neblina",
    12: "Nevoeiro",
    13: "Tempestade de areia",
    14: "Tempestade de granizo",
    15: "Chuvas fortes com granizo",
    51: "Chuva leve",
    53: "Chuva moderada",
    55: "Chuva forte",
    61: "Chuva leve com trovoadas",
    63: "Chuva moderada com trovoadas",
    65: "Chuva forte com trovoadas",
    71: "Neve leve",
    73: "Neve moderada",
    75: "Neve forte",
    81: "Chuva leve com granizo",
    82: "Chuva moderada com granizo",
    83: "Chuva forte com granizo",
    # Adicione mais códigos conforme necessário
}

# Faz a requisição GET à API
resposta = requests.get(url)

# Verifica se a requisição foi bem-sucedida
if resposta.status_code == 200:
    dados = resposta.json()

    # Dicionário para armazenar os códigos de tempo por mês
    weather_codes_mensais = defaultdict(list)

    # Itera sobre as datas e códigos de tempo diários
    for i in range(len(dados['daily']['time'])):
        data = dados['daily']['time'][i]
        weather_code = dados['daily']['weathercode'][i]

        # Extrai o mês da data
        mes = dt.strptime(data, "%Y-%m-%d").strftime("%Y-%m")

        # Adiciona o código de tempo à lista correspondente ao mês
        weather_codes_mensais[mes].append(weather_code)

    # Determina o tempo predominante em cada mês
    for mes, codes in weather_codes_mensais.items():
        # Conta a frequência de cada código de tempo no mês
        frequencia = Counter(codes)
        # Encontra o código de tempo mais comum
        tempo_predominante = frequencia.most_common(1)[0][0]
        # Converte o código de tempo para a descrição correspondente
        descricao_tempo = wmo_codes.get(tempo_predominante, "Código desconhecido")
        print(f"Tempo predominante em {mes}: {descricao_tempo} (WMO Code {tempo_predominante})")
else:
    print(f"Erro ao acessar a API: {resposta.status_code}")

Tempo predominante em 2024-01: Chuva moderada com trovoadas (WMO Code 63)
Tempo predominante em 2024-02: Chuva leve (WMO Code 51)
Tempo predominante em 2024-03: Chuva leve (WMO Code 51)
Tempo predominante em 2024-04: Nublado (WMO Code 3)
Tempo predominante em 2024-05: Chuva leve (WMO Code 51)
Tempo predominante em 2024-06: Céu limpo com poucas nuvens (WMO Code 1)
Tempo predominante em 2024-07: Nublado (WMO Code 3)
Tempo predominante em 2024-08: Parcialmente nublado (WMO Code 2)


Resposta:

Tempo predominante em 2024-01: Chuva moderada com trovoadas (WMO Code 63)

Tempo predominante em 2024-02: Chuva leve (WMO Code 51)

Tempo predominante em 2024-03: Chuva leve (WMO Code 51)

Tempo predominante em 2024-04: Nublado (WMO Code 3)

Tempo predominante em 2024-05: Chuva leve (WMO Code 51)

Tempo predominante em 2024-06: Céu limpo com poucas nuvens (WMO Code 1)

Tempo predominante em 2024-07: Nublado (WMO Code 3)

Tempo predominante em 2024-08: Parcialmente nublado (WMO Code 2)

#### 6. **Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?**

In [ ]:
# Função para fazer requisições GET
def get(url):
    resposta = requests.get(url)
    return resposta

# Função para verificar se a data é um dia de semana
def verifica_dia_da_semana(ui_data: str) -> bool:
    data = dt.strptime(ui_data, "%Y-%m-%d")
    return 0 <= data.weekday() <= 4

# Coordenadas do Rio de Janeiro
latitude = -22.9064
longitude = -43.1822

# URL da API Open-Meteo
weather_url = f"https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date=2024-01-01&end_date=2024-08-01&daily=temperature_2m_max,temperature_2m_min,weathercode&timezone=America/Sao_Paulo"

# URL da API de Feriados
holidays_url = "https://date.nager.at/api/v3/publicholidays/2024/BR"

# Tabela de códigos WMO
wmo_codes = {
    0: "Céu limpo",
    1: "Céu limpo com poucas nuvens",
    2: "Parcialmente nublado",
    3: "Nublado",
    4: "Nublado com chuvas leves",
    5: "Chuva moderada",
    6: "Chuva forte",
    7: "Chuva intensa com trovoadas",
    8: "Neve leve",
    9: "Neve moderada",
    10: "Neve forte",
    11: "Neblina",
    12: "Nevoeiro",
    13: "Tempestade de areia",
    14: "Tempestade de granizo",
    15: "Chuvas fortes com granizo",
    51: "Chuva leve",
    53: "Chuva moderada",
    55: "Chuva forte",
    61: "Chuva leve com trovoadas",
    63: "Chuva moderada com trovoadas",
    65: "Chuva forte com trovoadas",
    71: "Neve leve",
    73: "Neve moderada",
    75: "Neve forte",
    81: "Chuva leve com granizo",
    82: "Chuva moderada com granizo",
    83: "Chuva forte com granizo",
}

# Obtém os dados dos feriados
feriados_resposta = get(holidays_url)
feriados = feriados_resposta.json()

# Obtém os dados meteorológicos
weather_resposta = get(weather_url)
weather_dados = weather_resposta.json()

# Dicionário para armazenar as temperaturas e o tempo por data
dados_feriados = {}

# Preenche o dicionário com os feriados e suas datas
for feriado in feriados:
    data_feriado = feriado['date']
    dados_feriados[data_feriado] = {'temperaturas': [], 'weathercodes': []}

# Preenche o dicionário com as temperaturas e os códigos de tempo diários
for i in range(len(weather_dados['daily']['time'])):
    data = weather_dados['daily']['time'][i]
    temp_max = weather_dados['daily']['temperature_2m_max'][i]
    temp_min = weather_dados['daily']['temperature_2m_min'][i]
    weather_code = weather_dados['daily']['weathercode'][i]

    if data in dados_feriados:
        dados_feriados[data]['temperaturas'].append((temp_max + temp_min) / 2)
        dados_feriados[data]['weathercodes'].append(weather_code)

# Calcula a temperatura média e o tempo para cada feriado
for data_feriado, info in dados_feriados.items():
    if info['temperaturas']:
        temperatura_media = sum(info['temperaturas']) / len(info['temperaturas'])
        tempo_predominante = Counter(info['weathercodes']).most_common(1)[0][0]
        descricao_tempo = wmo_codes.get(tempo_predominante, "Código desconhecido")
        print(f"Feriado em {data_feriado}:")
        print(f"  Temperatura média: {temperatura_media:.2f}°C")
        print(f"  Tempo predominante: {descricao_tempo} (WMO Code {tempo_predominante})")
    else:
        print(f"Feriado em {data_feriado}: Dados meteorológicos não disponíveis")

Feriado em 2024-01-01:
  Temperatura média: 24.85°C
  Tempo predominante: Chuva leve (WMO Code 51)
Feriado em 2024-02-12:
  Temperatura média: 30.80°C
  Tempo predominante: Céu limpo com poucas nuvens (WMO Code 1)
Feriado em 2024-02-13:
  Temperatura média: 31.15°C
  Tempo predominante: Chuva leve (WMO Code 51)
Feriado em 2024-03-29:
  Temperatura média: 25.70°C
  Tempo predominante: Chuva leve com trovoadas (WMO Code 61)
Feriado em 2024-03-31:
  Temperatura média: 24.70°C
  Tempo predominante: Chuva moderada (WMO Code 53)
Feriado em 2024-04-21:
  Temperatura média: 23.50°C
  Tempo predominante: Parcialmente nublado (WMO Code 2)
Feriado em 2024-05-01:
  Temperatura média: 28.35°C
  Tempo predominante: Céu limpo (WMO Code 0)
Feriado em 2024-05-30:
  Temperatura média: 20.90°C
  Tempo predominante: Chuva leve (WMO Code 51)
Feriado em 2024-07-09:
  Temperatura média: 21.95°C
  Tempo predominante: Chuva moderada (WMO Code 53)
Feriado em 2024-09-07: Dados meteorológicos não disponíveis
Feri

Resposta:

###Feriado em 2024-01-01 Ano Novo:

  Temperatura média: 24.85°C

  Tempo predominante: Chuva leve (WMO Code 51)

###Feriado em 2024-02-12 Carnaval:

  Temperatura média: 30.80°C

  Tempo predominante: Céu limpo com poucas nuvens (WMO Code 1)

###Feriado em 2024-02-13 Carnaval:

  Temperatura média: 31.15°C

  Tempo predominante: Chuva leve (WMO Code 51)

###Feriado em 2024-03-29 Sexta-feira Santa:

  Temperatura média: 25.70°C

  Tempo predominante: Chuva leve com trovoadas (WMO Code 61)

###Feriado em 2024-03-31 Páscoa:

  Temperatura média: 24.70°C

  Tempo predominante: Chuva moderada (WMO Code 53)

###Feriado em 2024-04-21 Tiradentes:

  Temperatura média: 23.50°C

  Tempo predominante: Parcialmente nublado (WMO Code 2)

###Feriado em 2024-05-01 Dia do Trabalho:

  Temperatura média: 28.35°C

  Tempo predominante: Céu limpo (WMO Code 0)

###Feriado em 2024-05-30 Corpus Christi:

  Temperatura média: 20.90°C

  Tempo predominante: Chuva leve (WMO Code 51)

###Feriado em 2024-07-09:
(Em 2024, o dia 9 de julho não é feriado nacional, mas é feriado estadual em São Paulo, onde se comemora o Dia da Revolução Constitucionalista. Fiquei em dúvida pq a questão não cita nacional)

  Temperatura média: 21.95°C

  Tempo predominante: Chuva moderada (WMO Code 53)

#### 7. **Considere as seguintes suposições:**
   - O cidadão carioca considera "frio" um dia cuja temperatura média é menor que 20ºC;
   - Um feriado bem aproveitado no Rio de Janeiro é aquele em que se pode ir à praia;
   - O cidadão carioca só vai à praia quando não está com frio;
   - O cidadão carioca também só vai à praia em dias com sol, evitando dias **totalmente** nublados ou chuvosos (considere _weather_code_ para determinar as condições climáticas).

   Houve algum feriado "não aproveitável" em 2024? Se sim, qual(is)?

In [ ]:
# Função para fazer requisições GET
def get(url):
    resposta = requests.get(url)
    return resposta

# Coordenadas do Rio de Janeiro
latitude = -22.9064
longitude = -43.1822

# URL da API Open-Meteo
weather_url = f"https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date=2024-01-01&end_date=2024-08-01&daily=temperature_2m_max,temperature_2m_min,weathercode&timezone=America/Sao_Paulo"

# URL da API de Feriados
holidays_url = "https://date.nager.at/api/v3/publicholidays/2024/BR"

# Tabela de códigos WMO
wmo_codes = {
    0: "Céu limpo",
    1: "Céu limpo com poucas nuvens",
    2: "Parcialmente nublado",
    3: "Nublado",
    4: "Nublado com chuvas leves",
    5: "Chuva moderada",
    6: "Chuva forte",
    7: "Chuva intensa com trovoadas",
    8: "Neve leve",
    9: "Neve moderada",
    10: "Neve forte",
    11: "Neblina",
    12: "Nevoeiro",
    13: "Tempestade de areia",
    14: "Tempestade de granizo",
    15: "Chuvas fortes com granizo",
    51: "Chuva leve",
    53: "Chuva moderada",
    55: "Chuva forte",
    61: "Chuva leve com trovoadas",
    63: "Chuva moderada com trovoadas",
    65: "Chuva forte com trovoadas",
    71: "Neve leve",
    73: "Neve moderada",
    75: "Neve forte",
    81: "Chuva leve com granizo",
    82: "Chuva moderada com granizo",
    83: "Chuva forte com granizo",
}

# Obtém os dados dos feriados
feriados_resposta = get(holidays_url)
feriados = feriados_resposta.json()

# Obtém os dados meteorológicos
weather_resposta = get(weather_url)
weather_dados = weather_resposta.json()

# Dicionário para armazenar as temperaturas e o tempo por data
dados_feriados = {}

# Preenche o dicionário com os feriados e suas datas
for feriado in feriados:
    data_feriado = feriado['date']
    dados_feriados[data_feriado] = {'temperaturas': [], 'weathercodes': []}

# Preenche o dicionário com as temperaturas e os códigos de tempo diários
for i in range(len(weather_dados['daily']['time'])):
    data = weather_dados['daily']['time'][i]
    temp_max = weather_dados['daily']['temperature_2m_max'][i]
    temp_min = weather_dados['daily']['temperature_2m_min'][i]
    weather_code = weather_dados['daily']['weathercode'][i]

    if data in dados_feriados:
        dados_feriados[data]['temperaturas'].append((temp_max + temp_min) / 2)
        dados_feriados[data]['weathercodes'].append(weather_code)

# Função para identificar feriados não aproveitáveis
def non_enjoyable_holidays(holiday_weather_info):
    non_enjoyable = []
    for date, info in holiday_weather_info.items():
        if info['temperaturas']:
            temperatura_media = sum(info['temperaturas']) / len(info['temperaturas'])
            tempo_predominante = Counter(info['weathercodes']).most_common(1)[0][0]
            if temperatura_media < 20 or tempo_predominante not in [0, 1, 2, 3]:
                non_enjoyable.append({
                    "data": date,
                    "temperatura_média": temperatura_media,
                    "tempo_predominante": wmo_codes.get(tempo_predominante, "Código desconhecido")
                })
    return non_enjoyable

# Obtém os feriados não aproveitáveis
feriados_nao_aproveitaveis = non_enjoyable_holidays(dados_feriados)

# Exibe os resultados
if feriados_nao_aproveitaveis:
    print("Feriados não aproveitáveis em 2024:")
    for feriado in feriados_nao_aproveitaveis:
        print(f"Data: {feriado['data']}, Temperatura média: {feriado['temperatura_média']:.2f}°C, Tempo predominante: {feriado['tempo_predominante']}")
else:
    print("Não houve feriados não aproveitáveis em 2024.")


Feriados não aproveitáveis em 2024:
Data: 2024-01-01, Temperatura média: 24.85°C, Tempo predominante: Chuva leve
Data: 2024-02-13, Temperatura média: 31.15°C, Tempo predominante: Chuva leve
Data: 2024-03-29, Temperatura média: 25.70°C, Tempo predominante: Chuva leve com trovoadas
Data: 2024-03-31, Temperatura média: 24.70°C, Tempo predominante: Chuva moderada
Data: 2024-05-30, Temperatura média: 20.90°C, Tempo predominante: Chuva leve
Data: 2024-07-09, Temperatura média: 21.95°C, Tempo predominante: Chuva moderada


Resposta: Foram 6 Feriados não aproveitáveis em 2024:

Data: 2024-01-01, Temperatura média: 24.85°C, Tempo predominante: Chuva leve

Data: 2024-02-13, Temperatura média: 31.15°C, Tempo predominante: Chuva leve

Data: 2024-03-29, Temperatura média: 25.70°C, Tempo predominante: Chuva leve com trovoadas

Data: 2024-03-31, Temperatura média: 24.70°C, Tempo predominante: Chuva moderada

Data: 2024-05-30, Temperatura média: 20.90°C, Tempo predominante: Chuva leve

Data: 2024-07-09, Temperatura média: 21.95°C, Tempo predominante: Chuva moderada

#### 8. **Qual foi o feriado "mais aproveitável" de 2024?**  
   Considere o melhor par tempo e temperatura.

In [ ]:
# Função para fazer requisições GET
def get(url):
    resposta = requests.get(url)
    return resposta

# Coordenadas do Rio de Janeiro
latitude = -22.9064
longitude = -43.1822

# URL da API Open-Meteo
weather_url = f"https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date=2024-01-01&end_date=2024-08-01&daily=temperature_2m_max,temperature_2m_min,weathercode&timezone=America/Sao_Paulo"

# URL da API de Feriados
holidays_url = "https://date.nager.at/api/v3/publicholidays/2024/BR"

# Tabela de códigos WMO
wmo_codes = {
    0: "Céu limpo",
    1: "Céu limpo com poucas nuvens",
    2: "Parcialmente nublado",
    3: "Nublado",
    4: "Nublado com chuvas leves",
    5: "Chuva moderada",
    6: "Chuva forte",
    7: "Chuva intensa com trovoadas",
    8: "Neve leve",
    9: "Neve moderada",
    10: "Neve forte",
    11: "Neblina",
    12: "Nevoeiro",
    13: "Tempestade de areia",
    14: "Tempestade de granizo",
    15: "Chuvas fortes com granizo",
    51: "Chuva leve",
    53: "Chuva moderada",
    55: "Chuva forte",
    61: "Chuva leve com trovoadas",
    63: "Chuva moderada com trovoadas",
    65: "Chuva forte com trovoadas",
    71: "Neve leve",
    73: "Neve moderada",
    75: "Neve forte",
    81: "Chuva leve com granizo",
    82: "Chuva moderada com granizo",
    83: "Chuva forte com granizo",
}

# Obtém os dados dos feriados
feriados_resposta = get(holidays_url)
feriados = feriados_resposta.json()

# Obtém os dados meteorológicos
weather_resposta = get(weather_url)
weather_dados = weather_resposta.json()

# Dicionário para armazenar as temperaturas e o tempo por data
dados_feriados = {}

# Preenche o dicionário com os feriados e suas datas
for feriado in feriados:
    data_feriado = feriado['date']
    dados_feriados[data_feriado] = {'temperaturas': [], 'weathercodes': []}

# Preenche o dicionário com as temperaturas e os códigos de tempo diários
for i in range(len(weather_dados['daily']['time'])):
    data = weather_dados['daily']['time'][i]
    temp_max = weather_dados['daily']['temperature_2m_max'][i]
    temp_min = weather_dados['daily']['temperature_2m_min'][i]
    weather_code = weather_dados['daily']['weathercode'][i]

    if data in dados_feriados:
        dados_feriados[data]['temperaturas'].append((temp_max + temp_min) / 2)
        dados_feriados[data]['weathercodes'].append(weather_code)

# Função para identificar o feriado mais aproveitável
def best_holiday(holiday_weather_info):
    melhor_feriado = None
    melhor_criterio = -float('inf')  # Critério para maximizar

    for date, info in holiday_weather_info.items():
        if info['temperaturas']:
            temperatura_media = sum(info['temperaturas']) / len(info['temperaturas'])
            tempo_predominante = Counter(info['weathercodes']).most_common(1)[0][0]

            # Critério para avaliar o feriado
            if temperatura_media >= 20 and tempo_predominante in [0, 1, 2, 3]:
                criterio = temperatura_media  # Aqui, usamos a temperatura como critério principal

                if criterio > melhor_criterio:
                    melhor_criterio = criterio
                    melhor_feriado = {
                        "data": date,
                        "temperatura_média": temperatura_media,
                        "tempo_predominante": wmo_codes.get(tempo_predominante, "Código desconhecido")
                    }

    return melhor_feriado

# Obtém o feriado mais aproveitável
feriado_mais_aproveitavel = best_holiday(dados_feriados)

# Exibe o resultado
if feriado_mais_aproveitavel:
    print(f"O feriado mais aproveitável em 2024 é:")
    print(f"Data: {feriado_mais_aproveitavel['data']}, Temperatura média: {feriado_mais_aproveitavel['temperatura_média']:.2f}°C, Tempo predominante: {feriado_mais_aproveitavel['tempo_predominante']}")
else:
    print("Não houve feriado aproveitável em 2024.")
############################################

O feriado mais aproveitável em 2024 é:
Data: 2024-02-12, Temperatura média: 30.80°C, Tempo predominante: Céu limpo com poucas nuvens


Resposta:

 O feriado do carnaval foi o mais aproveitável em 2024:

Data: 2024-02-12,

Temperatura média: 30.80°C,

Tempo predominante: Céu limpo com poucas nuvens